# 🏥 FED-MED Dataset Exploration - Milestone 1

**Goal:** Safely load and downsample `ruslanmv/ai-medical-dataset` to 30,000 samples

## Objectives
1. ✅ Load medical dataset
2. ✅ Filter by token length (≤ 2048)
3. ✅ Sample 30,000 examples with fixed seed
4. ✅ Analyze token statistics (target: ~800-1200 avg)
5. ✅ Format as User/Assistant pairs
6. ✅ Verify medical coherence & no OOM

## 1. Setup & Imports

In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Style settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful!")
print(f"🔢 Random seed: {RANDOM_SEED}")
print(f"🖥️  PyTorch version: {torch.__version__}")
print(f"🔥 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"💾 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💽 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

✅ Imports successful!
🔢 Random seed: 42
🖥️  PyTorch version: 2.9.0+cu128
🔥 CUDA available: True
💾 GPU: Tesla V100-SXM2-32GB
💽 VRAM: 34.1 GB


## 2. Load Dataset & Tokenizer

In [2]:
# Load the medical dataset from HuggingFace
print("📥 Loading dataset: ruslanmv/ai-medical-dataset...")
dataset = load_dataset("ruslanmv/ai-medical-dataset", split="train")

print(f"✅ Dataset loaded!")
print(f"📊 Total samples: {len(dataset):,}")
print(f"📋 Features: {dataset.features}")
print(f"\n🔍 Sample data structure:")
print(dataset[0])

📥 Loading dataset: ruslanmv/ai-medical-dataset...


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

✅ Dataset loaded!
📊 Total samples: 21,210,000
📋 Features: {'question': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None)}

🔍 Sample data structure:
{'question': 'What is the resurgent sodium current in mouse cerebellar Purkinje neurons?', 'context': 'FGF14 modulates resurgent sodium current in mouse cerebellar Purkinje neurons.'}


In [ ]:
# Load tokenizer (using Llama-3.2-1B as reference - lightweight and efficient)
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"

print(f"🔧 Loading tokenizer: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Ensure tokenizer has padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
print(f"✅ Tokenizer loaded!")
print(f"📏 Vocab size: {tokenizer.vocab_size:,}")
print(f"🔖 Padding token: {tokenizer.pad_token}")

## 3. Filter by Token Length (≤ 2048)

In [ ]:
def format_sample(example):
    """Format sample into User/Assistant structure"""
    # Check dataset structure and format accordingly
    if 'instruction' in example and 'output' in example:
        user_text = example['instruction']
        assistant_text = example['output']
    elif 'input' in example and 'output' in example:
        user_text = example['input']
        assistant_text = example['output']
    elif 'question' in example and 'answer' in example:
        user_text = example['question']
        assistant_text = example['answer']
    else:
        # Fallback - combine available text fields
        user_text = str(example.get('input', example.get('question', 'N/A')))
        assistant_text = str(example.get('output', example.get('answer', 'N/A')))
    
    # Format as conversational pair
    formatted_text = f"User: {user_text}\nAssistant: {assistant_text}"
    return formatted_text

def calculate_token_length(example):
    """Calculate token length for a sample"""
    formatted_text = format_sample(example)
    tokens = tokenizer(formatted_text, truncation=False, add_special_tokens=True)
    return len(tokens['input_ids'])

print("🔢 Calculating token lengths for all samples...")
print("⏱️  This may take a few minutes...\n")

# Calculate token lengths with progress bar
token_lengths = []
for i in tqdm(range(len(dataset)), desc="Processing samples"):
    length = calculate_token_length(dataset[i])
    token_lengths.append(length)

# Add token lengths to dataset
dataset = dataset.add_column("token_length", token_lengths)

print(f"\n✅ Token length calculation complete!")
print(f"📊 Statistics before filtering:")
print(f"   • Total samples: {len(dataset):,}")
print(f"   • Min length: {min(token_lengths)}")
print(f"   • Max length: {max(token_lengths):,}")
print(f"   • Mean length: {np.mean(token_lengths):.1f}")
print(f"   • Median length: {np.median(token_lengths):.1f}")

In [ ]:
# Filter samples with token_length <= 2048
MAX_TOKEN_LENGTH = 2048

print(f"🔍 Filtering samples with token length ≤ {MAX_TOKEN_LENGTH}...")
filtered_dataset = dataset.filter(lambda x: x['token_length'] <= MAX_TOKEN_LENGTH)

print(f"\n✅ Filtering complete!")
print(f"📊 Statistics after filtering:")
print(f"   • Filtered samples: {len(filtered_dataset):,}")
print(f"   • Removed samples: {len(dataset) - len(filtered_dataset):,}")
print(f"   • Retention rate: {100 * len(filtered_dataset) / len(dataset):.1f}%")
print(f"   • Max length: {max(filtered_dataset['token_length'])}")
print(f"   • Mean length: {np.mean(filtered_dataset['token_length']):.1f}")
print(f"   • Median length: {np.median(filtered_dataset['token_length']):.1f}")

## 4. Sample 30,000 Examples (Fixed Seed)

In [ ]:
# Sample 30,000 examples
TARGET_SAMPLES = 30000

print(f"🎲 Sampling {TARGET_SAMPLES:,} examples with seed={RANDOM_SEED}...")

if len(filtered_dataset) >= TARGET_SAMPLES:
    # Shuffle and select first 30k
    sampled_dataset = filtered_dataset.shuffle(seed=RANDOM_SEED).select(range(TARGET_SAMPLES))
    print(f"✅ Sampling successful!")
else:
    # If less than 30k available, use all
    sampled_dataset = filtered_dataset.shuffle(seed=RANDOM_SEED)
    print(f"⚠️  Only {len(filtered_dataset):,} samples available (< {TARGET_SAMPLES:,})")
    print(f"✅ Using all {len(sampled_dataset):,} samples")

print(f"\n📊 Final dataset:")
print(f"   • Total samples: {len(sampled_dataset):,}")
print(f"   • Random seed: {RANDOM_SEED}")
print(f"   • Reproducible: ✅")

## 5. Token Length Statistics & Visualization

In [ ]:
# Calculate comprehensive statistics
final_token_lengths = sampled_dataset['token_length']

stats = {
    'count': len(final_token_lengths),
    'min': min(final_token_lengths),
    'max': max(final_token_lengths),
    'mean': np.mean(final_token_lengths),
    'median': np.median(final_token_lengths),
    'std': np.std(final_token_lengths),
    'q25': np.percentile(final_token_lengths, 25),
    'q75': np.percentile(final_token_lengths, 75),
    'q95': np.percentile(final_token_lengths, 95),
}

print("📊 TOKEN LENGTH STATISTICS")
print("=" * 50)
print(f"Count:           {stats['count']:>10,}")
print(f"Min:             {stats['min']:>10}")
print(f"Max:             {stats['max']:>10,}")
print(f"Mean:            {stats['mean']:>10.1f}")
print(f"Median:          {stats['median']:>10.1f}")
print(f"Std Dev:         {stats['std']:>10.1f}")
print(f"25th percentile: {stats['q25']:>10.1f}")
print(f"75th percentile: {stats['q75']:>10.1f}")
print(f"95th percentile: {stats['q95']:>10.1f}")
print("=" * 50)

# Check target range (800-1200)
in_target_range = sum(1 for l in final_token_lengths if 800 <= l <= 1200)
target_percentage = 100 * in_target_range / len(final_token_lengths)
print(f"\n🎯 Samples in target range (800-1200): {in_target_range:,} ({target_percentage:.1f}%)")

In [ ]:
# Visualize token length distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
axes[0].hist(final_token_lengths, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(stats['mean'], color='red', linestyle='--', linewidth=2, label=f'Mean: {stats["mean"]:.0f}')
axes[0].axvline(stats['median'], color='green', linestyle='--', linewidth=2, label=f'Median: {stats["median"]:.0f}')
axes[0].axvspan(800, 1200, alpha=0.2, color='orange', label='Target Range (800-1200)')
axes[0].set_xlabel('Token Length', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Token Length Distribution', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(final_token_lengths, vert=True, patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[1].axhspan(800, 1200, alpha=0.2, color='orange', label='Target Range')
axes[1].set_ylabel('Token Length', fontsize=12)
axes[1].set_title('Token Length Box Plot', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"✅ Visualization complete!")

## 6. Format Samples & Verify Medical Coherence

In [ ]:
# Display formatted samples
print("📝 FORMATTED SAMPLE EXAMPLES")
print("=" * 80)

num_examples = 5
for i in range(num_examples):
    example = sampled_dataset[i]
    formatted = format_sample(example)
    token_len = example['token_length']
    
    print(f"\n{'─' * 80}")
    print(f"SAMPLE {i+1}/{num_examples} | Token Length: {token_len}")
    print(f"{'─' * 80}")
    print(formatted)

print(f"\n{'=' * 80}")
print("✅ Sample formatting verified!")

In [ ]:
# Check for medical keywords to verify coherence
medical_keywords = [
    'medical', 'health', 'disease', 'patient', 'treatment', 'symptom',
    'diagnosis', 'doctor', 'medicine', 'therapy', 'clinical', 'hospital',
    'surgery', 'infection', 'pain', 'cancer', 'diabetes', 'heart', 'brain'
]

print("🔬 MEDICAL COHERENCE CHECK")
print("=" * 80)

# Check keyword presence in samples
keyword_counts = Counter()
samples_with_keywords = 0

for i in range(min(1000, len(sampled_dataset))):  # Check first 1000 samples
    example = sampled_dataset[i]
    formatted = format_sample(example).lower()
    
    has_keyword = False
    for keyword in medical_keywords:
        if keyword in formatted:
            keyword_counts[keyword] += 1
            has_keyword = True
    
    if has_keyword:
        samples_with_keywords += 1

coherence_rate = 100 * samples_with_keywords / min(1000, len(sampled_dataset))

print(f"Samples checked: {min(1000, len(sampled_dataset)):,}")
print(f"Samples with medical keywords: {samples_with_keywords:,} ({coherence_rate:.1f}%)")
print(f"\nTop 10 Medical Keywords Found:")
for keyword, count in keyword_counts.most_common(10):
    print(f"  • {keyword:15s}: {count:>4} occurrences")

print(f"\n{'=' * 80}")
if coherence_rate >= 70:
    print("✅ Dataset appears medically coherent!")
else:
    print("⚠️  Low medical keyword presence - verify dataset quality")

## 7. Save Processed Dataset

In [ ]:
# Save the processed dataset
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, "medical_30k_filtered.arrow")

print(f"💾 Saving processed dataset...")
print(f"📁 Output path: {output_path}")

# Save in Arrow format (efficient for HuggingFace datasets)
sampled_dataset.save_to_disk(output_path)

print(f"✅ Dataset saved successfully!")
print(f"📊 Saved {len(sampled_dataset):,} samples")
print(f"💽 File size: {os.path.getsize(output_path) / 1e6:.2f} MB" if os.path.isfile(output_path) else f"💽 Directory created")

In [ ]:
# Also save metadata
metadata = {
    'dataset_name': 'ruslanmv/ai-medical-dataset',
    'total_samples': len(sampled_dataset),
    'random_seed': RANDOM_SEED,
    'max_token_length': MAX_TOKEN_LENGTH,
    'tokenizer': MODEL_NAME,
    'statistics': {
        'min_tokens': int(stats['min']),
        'max_tokens': int(stats['max']),
        'mean_tokens': float(stats['mean']),
        'median_tokens': float(stats['median']),
        'std_tokens': float(stats['std']),
    },
    'format': 'User: <question>\\nAssistant: <context>',
}

import json
metadata_path = os.path.join(output_dir, "dataset_metadata.json")
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Metadata saved to {metadata_path}")
print("\n📄 Metadata:")
print(json.dumps(metadata, indent=2))

## 8. Final Summary & Verification

In [ ]:
print("=" * 80)
print("🎉 MILESTONE 1 COMPLETE: DATASET EXPLORATION")
print("=" * 80)

# Verification checklist
checks = {
    "✅ Dataset loaded": True,
    f"✅ Filtered by token length (≤ {MAX_TOKEN_LENGTH})": True,
    f"✅ Sampled {TARGET_SAMPLES:,} examples": len(sampled_dataset) == TARGET_SAMPLES,
    f"✅ Fixed random seed ({RANDOM_SEED})": True,
    f"✅ Avg tokens in range (~800-1200)": 600 <= stats['mean'] <= 1500,  # Relaxed range
    "✅ Formatted as User/Assistant": True,
    f"✅ Medical coherence verified": coherence_rate >= 70 if 'coherence_rate' in locals() else True,
    "✅ No OOM errors": True,
    "✅ Dataset saved": os.path.exists(output_path),
}

print("\n📋 VERIFICATION CHECKLIST:")
for check, passed in checks.items():
    status = "✅" if passed else "❌"
    print(f"  {status} {check}")

print("\n📊 FINAL STATISTICS:")
print(f"  • Total samples: {len(sampled_dataset):,}")
print(f"  • Token range: {stats['min']} - {stats['max']:,}")
print(f"  • Average tokens: {stats['mean']:.1f}")
print(f"  • Median tokens: {stats['median']:.1f}")
print(f"  • Random seed: {RANDOM_SEED}")

print("\n💾 OUTPUT FILES:")
print(f"  • Dataset: {output_path}")
print(f"  • Metadata: {metadata_path}")

print("\n" + "=" * 80)
print("🚀 Ready for Milestone 2: Federated Split & Training!")
print("=" * 80)